<a href="https://colab.research.google.com/github/brokencranium/DSA/blob/master/Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: May 5th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [0]:
!pip install -q gpt_2_simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## Verify GPU

Colaboratory now uses an Nvidia T4 GPU, which is slightly faster than the old Nvidia K80 GPU for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text. However sometimes the K80 will still be used.

You can verify which GPU is active by running the cell below.

In [0]:
!nvidia-smi

Sun May  5 16:40:56 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    75W / 149W |   8339MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are two sizes of GPT-2:

* `117M` (default): the "small" model, 500MB on disk.
* `345M`: the "medium" model, 1.5GB on disk.

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [0]:
gpt2.download_gpt2(model_name="117M")

Fetching checkpoint: 1.00kit [00:00, 319kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 53.1Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 503kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:06, 75.6Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 1.74Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 42.1Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 40.9Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [0]:
gpt2.mount_gdrive()

## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
file_name = "shakespeare.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='117M',
              steps=1000,
              restore_from='fresh',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Loading checkpoint models/117M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/117M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.05s/it]


dataset has 338025 tokens
Training...
[10 | 29.42] loss=3.66 avg=3.66
[20 | 53.33] loss=3.83 avg=3.75
[30 | 76.63] loss=3.48 avg=3.66
[40 | 99.91] loss=3.23 avg=3.55
[50 | 123.47] loss=3.50 avg=3.54
[60 | 147.03] loss=3.42 avg=3.52
[70 | 170.45] loss=3.28 avg=3.48
[80 | 193.86] loss=3.08 avg=3.43
[90 | 217.36] loss=3.29 avg=3.41
[100 | 240.86] loss=3.26 avg=3.40
[110 | 264.33] loss=3.02 avg=3.36
[120 | 287.79] loss=2.96 avg=3.33
[130 | 311.23] loss=3.01 avg=3.30
[140 | 334.68] loss=3.08 avg=3.28
[150 | 358.10] loss=2.80 avg=3.25
[160 | 381.53] loss=3.03 avg=3.23
[170 | 404.94] loss=2.95 avg=3.22
[180 | 428.36] loss=3.10 avg=3.21
[190 | 451.82] loss=2.83 avg=3.19
[200 | 475.25] loss=2.86 avg=3.17
======== SAMPLE 1 ========
 no of them that I mean nor any they seem: they are all ungentles; they are all noisier, being noisier: they are all but a strange kind of sensual pleasure.

PRINCE:
So that, having these thoughts I take them as I have made the best and most perfect,
These will be my 

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. (NB: if you are downloading the model to your personal computer, download the large model checkpoint file *seperately*, download the other files, and reconstruct the `/checkpoint/run1` folder hierarchy locally).

In [0]:
gpt2.copy_checkpoint_to_gdrive()

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `checkpoint` folder from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive()

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
gpt2.generate(sess)

I have sat here all day
Unto this hand, as if it
Did some thing nigh me, whoo!

DUCHESS OF YORK:
What hand?

QUEEN ELIZABETH:
My uncle Clarence' greatest kindness.

DUCHESS OF YORK:
My liege, I had rather lie in wait on thee.
I promised thy duke should come and apprehend
Thy wife and young daughter.

DUCHESS OF YORK:
What, do you mean, to apprehend my daughter?

QUEEN ELIZABETH:
To answer the duke.

DUCHESS OF YORK:
Why, what then? 'tis time thou hadst come.
Uncle, you have made a sainted father of you.
Dare you break your fast before the king's seal
And besiege the buried foes with a holy vow?

QUEEN ELIZABETH:
Why, how now, uncle! hast thou forsook thy wife
For divines; for she's the mother of all
Abstinence; that she may enter
Of these most holy nuns?
Or blasphemers, perjured liars, and true
Of Christ so manifest?

DUCHESS OF YORK:
Have you forsook your wife
For any of these liars; for few,
When they shall come to speak for them, but they
Must heartily confess themselves true.

QUEE

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="LORD",
              nsamples=5,
              batch_size=5
              )

LORD FITZWATER:
Saw'st thou not, sir?

QUEEN ELIZABETH:
Madam, didst thou not mean, as some do,
To cram others with beggary?

QUEEN ELIZABETH:
Ha, ha! what should you say? why didst thou say no?

QUEEN ELIZABETH:
Hath this woman in this world hungrier than man?
Think'st thou that all these woes can be avoided?

QUEEN ELIZABETH:
As many lives she hath prevented as man.

DUCHESS OF YORK:
Why, hark! the Tower is brought down.

QUEEN ELIZABETH:
The King of Naples comes apace.

DUCHESS OF YORK:
It shall be so, I hope.

QUEEN ELIZABETH:
The Tower comes down.

DUCHESS OF YORK:
The Naples comes apace.

QUEEN ELIZABETH:
It shall be so, I hope.

DUCHESS OF YORK:


LORDY:
Every man that's gone hath died
And that's the end of all.

GLOUCESTER:
Your lordship must be revenged, Hermione.

LADY ANNE:
And she shall be the man that I dreamt of.

GLOUCESTER:
I cannot sleep: never a waking man shall
Go to any sleepder than to sleep.

LADY ANNE:
It shall be such a dream, I'll bear witness to it,
That I may

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp and then download it.

You can rerun the cell as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

files.download(gen_file)

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail or out-of-memory/OOM), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.